In [1]:
# q_dict = {'#': ['xmas', 'christmas']}
# tr = TwitterReader('./local_configs/api_keys.json')
# tr.set_query(q_dict)
# tweets = tr.read()['data']
# save_tweets(tweets)

In [2]:
# reset_db()

In [3]:
from data_access.twitter_reader import TwitterReader
from data_storage.db_manager import DBManager
from data_wrangling.wrangler import Wrangler
from data_transformers.normalizers import TextNormalizer
from data_transformers.vectorizer import OneHotVectorizer
from models.models import KMeansClusters
from nltk import pos_tag

In [4]:
db_manager = DBManager()
vectorizer = OneHotVectorizer()
g = db_manager.get_processed_tweets()

In [5]:
text_normalizer = TextNormalizer()

In [6]:
from sklearn.pipeline import Pipeline

In [7]:
model = Pipeline([
    ('norm', TextNormalizer()),
    ('vect', OneHotVectorizer()),
    ('clusters', KMeansClusters(k=2))
])

clusters = model.fit_transform(g)